In [1]:
import pandas as pd
import io, requests, os 
from bs4 import BeautifulSoup
from tqdm import tqdm
pd.set_option('display.max_columns', None)

output_folder = 'CovidOct25-2022'
if not os.path.exists(output_folder):
    os.makedirs(output_folder, exist_ok=True)

# PVI data

In [2]:
# read in fips codes
fips = pd.read_csv(
    "https://github.com/COVID19PVI/data/raw/master/Model11.2.1/data/Model_11.2.1_20200228_data.csv", 
    header=12, usecols=["name","casrn"]
).rename(columns={'name':'Name','casrn':'FIPS'})

# get the PVI data links
r = requests.get("https://github.com/COVID19PVI/data/tree/master/Model11.2.1")
soup = BeautifulSoup(r.content, 'html.parser')
links = [
    x.attrs["href"].replace("blob","raw")
    for x in soup.find_all('a', {'class':'js-navigation-open Link--primary'})[1:-1]
]

In [3]:
# combine data
all_data = []
read_from_local = True # Set to true if github isn't responding

if read_from_local:
    filenames = [link.split('/')[-1] for link in links]
    model_path = '../../data/Model11.2.1/' # PVI git repo cloned here in local machine
    for filename in tqdm(filenames):
        new_data = pd.read_csv(os.path.join(model_path, filename))
        new_data["date"] = filename.split("_")[2]
        all_data.append(new_data)

else:
    for link in tqdm(links):
        new_data = pd.read_csv("https://github.com"+link)
        new_data["date"] = link.split("_")[2]
        all_data.append(new_data)

all_data = pd.concat(all_data)
all_data["date"] = [f"{i[:4]}-{i[4:6]}-{i[6:]}" for i in all_data.date]
all_data.head()

100%|██████████| 973/973 [00:12<00:00, 75.43it/s]


,ToxPi Score,HClust Group,KMeans Group,Name,Source,Infection Rate: Transmissible Cases!25!0xcc3333ff,Infection Rate: Disease Spread!5!0xe64d4dff,Pop Concentration: Pop Mobility!10!0x57b757ff,Pop Concentration: Residential Density!10!0x5ced5cff,Intervention: Social Distancing!10!0x4258c9ff,Intervention: Testing!10!0x6079f7ff,Health & Environment: Pop Demographics!10!0x6b0b9eff,Health & Environment: Air Pollution!10!0x8e26c4ff,Health & Environment: Age Distribution!10!0x9a42c8ff,Health & Environment: Co-morbidities!10!0xb460e0ff,Health & Environment: Health Disparities!10!0xc885ecff,Health & Environment: Hospital Beds!5!0xdeb9f1ff,date
0,0.659739,1,1,"California, Solano","-121.9357,38.2683",1.000000,1.000000,0.717074,0.6905,1.000,0.504366,0.622993,0.479042,0.504195,0.236237,0.235189,0.514283,2020-02-28
1,0.624339,2,1,"California, Madera","-119.7666,37.2151",0.638652,1.000000,0.601609,0.8239,0.825,0.504366,0.605671,0.658683,0.469686,0.321223,0.590624,0.613680,2020-02-28
2,0.598916,1,1,"California, Santa Cruz","-122.0067,37.0558",0.817887,1.000000,0.704589,0.9035,1.000,0.504366,0.409785,0.269461,0.511724,0.100021,0.272557,0.531471,2020-02-28
3,0.590967,2,1,"California, Imperial","-115.3669,33.0393",0.591782,0.666667,0.585525,0.9869,0.750,0.504366,0.525758,0.628743,0.443774,0.270404,0.620412,0.516830,2020-02-28
4,0.578017,1,1,"California, Santa Clara","-121.697,37.231",0.830152,0.880000,0.793513,0.7310,1.000,0.504366,0.452019,0.353293,0.452613,0.060895,0.131427,0.461418,2020-02-28


In [4]:
drop_columns = [
    'Name', 'Source','date', 'ToxPi Score', 'HClust Group', 'KMeans Group', 'Hospital Beds',
    'Pop Mobility', 'Co-morbidities', 'Pop Demographics', 'Air Pollution', 'Residential Density', 'Testing'
]
for column in all_data.columns:
    column_name = column.split('!')[0]
    if ":" in column_name:
        column_name = column_name.split(": ")[1]

    if column_name in drop_columns:
        continue
    print(f"Saving {column_name}", end = "... ")
    mx = all_data.pivot(index="Name",columns="date", values=column).reset_index()
    mx = pd.merge(mx, fips, on="Name", how="left")
    mx.insert(0, "FIPS", mx.pop("FIPS"))
    mx.to_csv(f"{output_folder}/{column_name}.csv", index=False, float_format='%.4f')
    print(u'\u2705')

Saving Transmissible Cases... ✅
Saving Disease Spread... ✅
Saving Pop Mobility... ✅
Saving Residential Density... ✅
Saving Social Distancing... ✅
Saving Testing... ✅
Saving Pop Demographics... ✅
Saving Air Pollution... ✅
Saving Age Distribution... ✅
Saving Co-morbidities... ✅
Saving Health Disparities... ✅


In [5]:
all_data.columns

Index(['ToxPi Score', 'HClust Group', 'KMeans Group', 'Name', 'Source',
       'Infection Rate: Transmissible Cases!25!0xcc3333ff',
       'Infection Rate: Disease Spread!5!0xe64d4dff',
       'Pop Concentration: Pop Mobility!10!0x57b757ff',
       'Pop Concentration: Residential Density!10!0x5ced5cff',
       'Intervention: Social Distancing!10!0x4258c9ff',
       'Intervention: Testing!10!0x6079f7ff',
       'Health & Environment: Pop Demographics!10!0x6b0b9eff',
       'Health & Environment: Air Pollution!10!0x8e26c4ff',
       'Health & Environment: Age Distribution!10!0x9a42c8ff',
       'Health & Environment: Co-morbidities!10!0xb460e0ff',
       'Health & Environment: Health Disparities!10!0xc885ecff',
       'Health & Environment: Hospital Beds!5!0xdeb9f1ff', 'date'],
      dtype='object')

# Cases
Source [USAFacts](https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/). Data collection starts from 2020-01-22. Note that data from today can take 3 days to update in this file.

In [2]:
response = requests.get("https://static.usafacts.org/public/data/covid-19/covid_confirmed_usafacts.csv")

file_object = io.StringIO(response.content.decode('utf-8'))
df = pd.read_csv(file_object)
df.head()

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-10-16,2022-10-17,2022-10-18,2022-10-19,2022-10-20,2022-10-21,2022-10-22,2022-10-23,2022-10-24,2022-10-25
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,18452,18452,18452,18452,18480,18480,18480,18480,18480,18480
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,65819,65819,65819,65819,65895,65895,65895,65895,65895,65895
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,6910,6910,6910,6910,6926,6926,6926,6926,6926,6926
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,7547,7547,7547,7547,7560,7560,7560,7560,7560,7560


In [3]:
df.drop(columns=['County Name', 'State', 'StateFIPS'], inplace=True)
df.rename({'countyFIPS':'FIPS'}, axis=1, inplace=True)

df = df[(~df['FIPS'].isna()) & (df['FIPS']>0)]
df.fillna(0, inplace=True)
df.to_csv(f'{output_folder}/Cases.csv', index=False)

# Deaths
Source [USAFacts](https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/). Data collection starts from 2020-01-22.

In [4]:
response = requests.get("https://static.usafacts.org/public/data/covid-19/covid_deaths_usafacts.csv")
file_object = io.StringIO(response.content.decode('utf-8'))
df = pd.read_csv(file_object)
df.head()

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-10-16,2022-10-17,2022-10-18,2022-10-19,2022-10-20,2022-10-21,2022-10-22,2022-10-23,2022-10-24,2022-10-25
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,228,228,228,228,228,228,228,228,228,228
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,715,715,715,715,716,716,716,716,716,716
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,103,103,103,103,103,103,103,103,103,103
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,108,108,108,108,108,108,108,108,108,108


In [5]:
df.drop(columns=['County Name', 'State', 'StateFIPS'], inplace=True)
df.rename({'countyFIPS':'FIPS'}, axis=1, inplace=True)

df = df[(~df['FIPS'].isna()) & (df['FIPS']>0)]
df.fillna(0, inplace=True)
df.to_csv(f'{output_folder}/Deaths.csv', index=False)

# Vaccination
From [CDC vaccinations in the US counties](https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-County/8xkx-amqh). Data collection starts from 2020-12-13. Note that data from today can take 3 days to update in this file.

In [6]:
response = requests.get("https://data.cdc.gov/api/views/8xkx-amqh/rows.csv?accessType=DOWNLOAD")
file_object = io.StringIO(response.content.decode('utf-8'))
df = pd.read_csv(file_object, low_memory=False)
df.head()

,Date,FIPS,MMWR_week,Recip_County,Recip_State,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_5Plus,Administered_Dose1_Recip_5PlusPop_Pct,...,Booster_Doses_Vax_Pct_UR_Equity,Booster_Doses_12PlusVax_Pct_UR_Equity,Booster_Doses_18PlusVax_Pct_UR_Equity,Booster_Doses_65PlusVax_Pct_UR_Equity,Census2019,Census2019_5PlusPop,Census2019_5to17Pop,Census2019_12PlusPop,Census2019_18PlusPop,Census2019_65PlusPop
0,10/26/2022,16031,43,Cassia County,ID,97.7,12334.0,51.3,12322.0,55.8,...,6.0,6.0,6.0,6.0,24030.0,22097.0,5654.0,19113.0,16443.0,3473.0
1,10/26/2022,48345,43,Motley County,TX,98.9,470.0,39.2,470.0,41.2,...,7.0,7.0,7.0,6.0,1200.0,1141.0,178.0,1061.0,963.0,368.0
2,10/26/2022,38055,43,McLean County,ND,92.7,5412.0,57.3,5404.0,61.0,...,8.0,8.0,8.0,7.0,9450.0,8866.0,1494.0,8048.0,7372.0,2291.0
3,10/26/2022,17151,43,Pope County,IL,98.2,1859.0,44.5,1855.0,45.6,...,8.0,8.0,8.0,7.0,4177.0,4066.0,389.0,3902.0,3677.0,1183.0
4,10/26/2022,21209,43,Scott County,KY,94.1,37146.0,65.2,36907.0,69.2,...,3.0,3.0,4.0,3.0,57004.0,53310.0,10592.0,47684.0,42718.0,7050.0


In [7]:
df = df[df['FIPS']!='UNK']
df['FIPS'] = df['FIPS'].astype(int)
df = df[['Date', 'FIPS', 'Series_Complete_Pop_Pct']]

In [8]:
df['Date'] = pd.to_datetime(df['Date'])
# df = df.sort_values(by=['FIPS', 'Date']).reset_index(drop=True)
df.fillna(0, inplace=True)

In [10]:
vaccinationCompleteDose = df[['Date', 'FIPS', 'Series_Complete_Pop_Pct']].pivot_table(
    values='Series_Complete_Pop_Pct',
    index=['FIPS'],
    columns='Date'
).reset_index().rename_axis(None, axis=1)
vaccinationCompleteDose.head()

,FIPS,2020-12-13 00:00:00,2020-12-14 00:00:00,2020-12-15 00:00:00,2020-12-16 00:00:00,2020-12-17 00:00:00,2020-12-18 00:00:00,2020-12-19 00:00:00,2020-12-20 00:00:00,2020-12-21 00:00:00,...,2022-08-24 00:00:00,2022-08-31 00:00:00,2022-09-07 00:00:00,2022-09-14 00:00:00,2022-09-21 00:00:00,2022-09-28 00:00:00,2022-10-05 00:00:00,2022-10-12 00:00:00,2022-10-19 00:00:00,2022-10-26 00:00:00
0,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,45.0,45.0,45.1,45.1,45.1,45.2,45.3,45.3,45.4,45.5
1,1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,52.1,52.2,52.2,52.2,52.3,52.3,52.4,52.5,52.5,52.6
2,1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,46.6,46.7,46.7,46.7,46.7,46.7,46.8,46.8,46.9,46.9
3,1007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,35.8,35.8,35.8,35.8,35.8,35.9,35.9,36.0,36.0,36.0
4,1009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,32.4,32.4,32.4,32.5,32.5,32.5,32.6,32.6,32.6,32.6


In [11]:
vaccinationCompleteDose.to_csv(f'{output_folder}/Vaccination.csv', index=False)

In [ ]:
def valid_date(date):
    try:
        pd.to_datetime(date)
        return True
    except:
        return False

def fill_missing_dates(data):
    df = data.copy()
    df = df.fillna(method='ffill').fillna(0)
    # this assumes the dates are along the column axis
    date_columns = sorted([pd.to_datetime(col) for col in df.columns if valid_date(col)])
    non_date_columns = [col for col in df.columns if col not in date_columns]
    df.columns = non_date_columns + date_columns

    total_days = (date_columns[1] - date_columns[0]).days
    missing_dates = []

    previous_date = date_columns[0]
    for date in date_columns[1:]:
        steps = (date - previous_date).days
        if steps > 1:
            for step in range(1, steps):
                missing_dates.append(previous_date+pd.to_timedelta(step, unit='D'))
        previous_date = date

    print(f'Number of missing dates {len(len(missing_dates))}')

    df = df.reindex(columns = ['FIPS'] + sorted(date_columns + missing_dates))
    df = df.T.fillna(method='ffill').T
    return df

In [28]:
# this was added because after May 2022, 
# vaccination data from CDC started having random missing dates
df = fill_missing_dates(vaccinationCompleteDose)
df.head()

,FIPS,2020-12-13 00:00:00,2020-12-14 00:00:00,2020-12-15 00:00:00,2020-12-16 00:00:00,2020-12-17 00:00:00,2020-12-18 00:00:00,2020-12-19 00:00:00,2020-12-20 00:00:00,2020-12-21 00:00:00,2020-12-22 00:00:00,2020-12-23 00:00:00,2020-12-24 00:00:00,2020-12-25 00:00:00,2020-12-26 00:00:00,2020-12-27 00:00:00,2020-12-28 00:00:00,2020-12-29 00:00:00,2020-12-30 00:00:00,2020-12-31 00:00:00,2021-01-01 00:00:00,2021-01-02 00:00:00,2021-01-03 00:00:00,2021-01-04 00:00:00,2021-01-05 00:00:00,2021-01-06 00:00:00,2021-01-07 00:00:00,2021-01-08 00:00:00,2021-01-09 00:00:00,2021-01-10 00:00:00,2021-01-11 00:00:00,2021-01-12 00:00:00,2021-01-13 00:00:00,2021-01-14 00:00:00,2021-01-15 00:00:00,2021-01-16 00:00:00,2021-01-17 00:00:00,2021-01-18 00:00:00,2021-01-19 00:00:00,2021-01-20 00:00:00,2021-01-21 00:00:00,2021-01-22 00:00:00,2021-01-23 00:00:00,2021-01-24 00:00:00,2021-01-25 00:00:00,2021-01-26 00:00:00,2021-01-27 00:00:00,2021-01-28 00:00:00,2021-01-29 00:00:00,2021-01-30 00:00:00,2021-01-31 00:00:00,2021-02-01 00:00:00,2021-02-02 00:00:00,2021-02-03 00:00:00,2021-02-04 00:00:00,2021-02-05 00:00:00,2021-02-06 00:00:00,2021-02-07 00:00:00,2021-02-08 00:00:00,2021-02-09 00:00:00,2021-02-10 00:00:00,2021-02-11 00:00:00,2021-02-12 00:00:00,2021-02-13 00:00:00,2021-02-14 00:00:00,2021-02-15 00:00:00,2021-02-16 00:00:00,2021-02-17 00:00:00,2021-02-18 00:00:00,2021-02-19 00:00:00,2021-02-20 00:00:00,2021-02-21 00:00:00,2021-02-22 00:00:00,2021-02-23 00:00:00,2021-02-24 00:00:00,2021-02-25 00:00:00,2021-02-26 00:00:00,2021-02-27 00:00:00,2021-02-28 00:00:00,2021-03-01 00:00:00,2021-03-02 00:00:00,2021-03-03 00:00:00,2021-03-04 00:00:00,2021-03-05 00:00:00,2021-03-06 00:00:00,2021-03-07 00:00:00,2021-03-08 00:00:00,2021-03-09 00:00:00,2021-03-10 00:00:00,2021-03-11 00:00:00,2021-03-12 00:00:00,2021-03-13 00:00:00,2021-03-14 00:00:00,2021-03-15 00:00:00,2021-03-16 00:00:00,2021-03-17 00:00:00,2021-03-18 00:00:00,2021-03-19 00:00:00,2021-03-20 00:00:00,2021-03-21 00:00:00,2021-03-22 00:00:00,2021-03-23 00:00:00,2021-03-24 00:00:00,2021-03-25 00:00:00,2021-03-26 00:00:00,2021-03-27 00:00:00,2021-03-28 00:00:00,2021-03-29 00:00:00,2021-03-30 00:00:00,2021-03-31 00:00:00,2021-04-01 00:00:00,2021-04-02 00:00:00,2021-04-03 00:00:00,2021-04-04 00:00:00,2021-04-05 00:00:00,2021-04-06 00:00:00,2021-04-07 00:00:00,2021-04-08 00:00:00,2021-04-09 00:00:00,2021-04-10 00:00:00,2021-04-11 00:00:00,2021-04-12 00:00:00,2021-04-13 00:00:00,2021-04-14 00:00:00,2021-04-15 00:00:00,2021-04-16 00:00:00,2021-04-17 00:00:00,2021-04-18 00:00:00,2021-04-19 00:00:00,2021-04-20 00:00:00,2021-04-21 00:00:00,2021-04-22 00:00:00,2021-04-23 00:00:00,2021-04-24 00:00:00,2021-04-25 00:00:00,2021-04-26 00:00:00,2021-04-27 00:00:00,2021-04-28 00:00:00,2021-04-29 00:00:00,2021-04-30 00:00:00,2021-05-01 00:00:00,2021-05-02 00:00:00,2021-05-03 00:00:00,2021-05-04 00:00:00,2021-05-05 00:00:00,2021-05-06 00:00:00,2021-05-07 00:00:00,2021-05-08 00:00:00,2021-05-09 00:00:00,2021-05-10 00:00:00,2021-05-11 00:00:00,2021-05-12 00:00:00,2021-05-13 00:00:00,2021-05-14 00:00:00,2021-05-15 00:00:00,2021-05-16 00:00:00,2021-05-17 00:00:00,2021-05-18 00:00:00,2021-05-19 00:00:00,2021-05-20 00:00:00,2021-05-21 00:00:00,2021-05-22 00:00:00,2021-05-23 00:00:00,2021-05-24 00:00:00,2021-05-25 00:00:00,2021-05-26 00:00:00,2021-05-27 00:00:00,2021-05-28 00:00:00,2021-05-29 00:00:00,2021-05-30 00:00:00,2021-05-31 00:00:00,2021-06-01 00:00:00,2021-06-02 00:00:00,2021-06-03 00:00:00,2021-06-04 00:00:00,2021-06-05 00:00:00,2021-06-06 00:00:00,2021-06-07 00:00:00,2021-06-08 00:00:00,2021-06-09 00:00:00,2021-06-10 00:00:00,2021-06-11 00:00:00,2021-06-12 00:00:00,2021-06-13 00:00:00,2021-06-14 00:00:00,2021-06-15 00:00:00,2021-06-16 00:00:00,2021-06-17 00:00:00,2021-06-18 00:00:00,2021-06-19 00:00:00,2021-06-20 00:00:00,2021-06-21 00:00:00,2021-06-22 00:00:00,2021-06-23 00:00:00,2021-06-24 00:00:00,2021-06-25 00:00:00,2021-06-26 00:00:00,2021-06-27 00:00:00,2021-06-28 00:00:00,2021-06-29 00:00:00,2021-06-30 00:

In [29]:
df.to_csv(f'{output_folder}/Vaccination.csv', index=False)

# Age distribution
PVI dashboard has age distribution data from 2019. So we updated that with [this age distribution data](https://www.census.gov/programs-surveys/popest/technical-documentation/research/evaluation-estimates/2020-evaluation-estimates/2010s-county-detail.html) from 2020. The data codebook is [here](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2020/cc-est2020-agesex.pdf).

In [81]:
response = requests.get('https://www2.census.gov/programs-surveys/popest/datasets/2010-2020/counties/asrh/CC-EST2020-AGESEX-ALL.csv')
file_object = io.StringIO(response.content.decode('latin1'))
df = pd.read_csv(file_object, low_memory=False)

# in case the download by code is not working
# df = pd.read_csv(f'{output_folder}/CC-EST2020-AGESEX-ALL.csv', encoding='latin1', low_memory=False)

df = df[df['YEAR']==13]
df['Name'] = df['STNAME'] + ', ' + df['CTYNAME'].apply(lambda x: x.replace(' County', ''))
df['FIPS'] = df['STATE'] * 1000 + df['COUNTY']
df['AgeDist'] = df['AGE65PLUS_TOT'].astype(int) / df['POPESTIMATE'].astype(int)
df.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,POPESTIMATE,POPEST_MALE,POPEST_FEM,UNDER5_TOT,UNDER5_MALE,UNDER5_FEM,AGE513_TOT,AGE513_MALE,AGE513_FEM,AGE1417_TOT,AGE1417_MALE,AGE1417_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE16PLUS_TOT,AGE16PLUS_MALE,AGE16PLUS_FEM,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1544_TOT,AGE1544_MALE,AGE1544_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,AGE04_TOT,AGE04_MALE,AGE04_FEM,AGE59_TOT,AGE59_MALE,AGE59_FEM,AGE1014_TOT,AGE1014_MALE,AGE1014_FEM,AGE1519_TOT,AGE1519_MALE,AGE1519_FEM,AGE2024_TOT,AGE2024_MALE,AGE2024_FEM,AGE2529_TOT,AGE2529_MALE,AGE2529_FEM,AGE3034_TOT,AGE3034_MALE,AGE3034_FEM,AGE3539_TOT,AGE3539_MALE,AGE3539_FEM,AGE4044_TOT,AGE4044_MALE,AGE4044_FEM,AGE4549_TOT,AGE4549_MALE,AGE4549_FEM,AGE5054_TOT,AGE5054_MALE,AGE5054_FEM,AGE5559_TOT,AGE5559_MALE,AGE5559_FEM,AGE6064_TOT,AGE6064_MALE,AGE6064_FEM,AGE6569_TOT,AGE6569_MALE,AGE6569_FEM,AGE7074_TOT,AGE7074_MALE,AGE7074_FEM,AGE7579_TOT,AGE7579_MALE,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM,Name,FIPS,AgeDist
12,50,1,1,Alabama,Autauga County,13,56130,27231,28899,3346,1739,1607,6566,3366,3200,3114,1553,1561,4416,2267,2149,44657,21353,23304,43104,20573,22531,21567,10606,10961,14798,7166,7632,14848,7161,7687,9042,3979,5063,3346,1739,1607,3494,1785,1709,3833,1961,1872,3622,1825,1797,3147,1615,1532,3822,1893,1929,3636,1756,1880,3743,1818,1925,3597,1699,1898,3820,1844,1976,3675,1811,1864,3912,1915,1997,3441,1591,1850,2790,1351,1439,2386,1032,1354,1796,749,1047,1142,494,648,928,353,575,39.3,37.8,40.5,"Alabama, Autauga",1001,0.161090
26,50,1,3,Alabama,Baldwin County,13,227989,110420,117569,12092,6319,5773,24636,12572,12064,11536,5787,5749,16067,8055,8012,185376,88563,96813,179725,85742,93983,77312,38054,39258,52651,25693,26958,62077,29384,32693,48930,22610,26320,12092,6319,5773,13356,6874,6482,14222,7179,7043,13371,6684,6687,11290,5677,5613,12517,6254,6263,12865,6256,6609,13524,6579,6945,13745,6604,7141,14328,6941,7387,14519,6963,7556,16420,7756,8664,16810,7724,9086,15519,7146,8373,13806,6532,7274,9457,4503,4954,5607,2554,3053,4541,1875,2666,43.8,42.5,45.2,"Alabama, Baldwin",1003,0.214616
40,50,1,5,Alabama,Barbour County,13,24652,13018,11634,1315,673,642,2574,1275,1299,1164,593,571,1992,1162,830,20183,10777,9406,19599,10477,9122,9300,5525,3775,6441,3917,2524,6188,3234,2954,4978,2164,2814,1315,673,642,1351,659,692,1520,763,757,1394,726,668,1465,882,583,1762,1113,649,1648,1004,644,1559,899,660,1472,901,571,1552,869,683,1467,795,672,1659,827,832,1510,743,767,1499,689,810,1406,619,787,971,439,532,587,245,342,515,172,343,40.9,38.9,44.5,"Alabama, Barbour",1005,0.201931
54,50,1,7,Alabama,Bibb County,13,22199,11807,10392,1232,600,632,2177,1139,1038,1057,567,490,1735,1013,722,18284,9795,8489,17733,9501,8232,8753,5088,3665,6217,3642,2575,6009,3169,2840,3772,1677,2095,1232,600,632,1199,628,571,1234,645,589,1224,675,549,1312,771,541,1704,1001,703,1607,962,645,1508,888,620,1398,791,607,1605,885,720,1492,796,696,1543,805,738,1369,683,686,1162,551,611,1047,492,555,700,310,390,467,188,279,396,136,260,40.3,38.4,42.9,"Alabama, Bibb",1007,0.169918
68,50,1,9,Alabama,Blount County,13,57932,28636,29296,3453,1798,1655,6674,3363,3311,3118,1525,1593,4440,2347,2093,46253,22734,23519,44687,21950,22737,20732,10473,10259,13939,6972,6967,15326,7652,7674,10982,4979,6003,3453,1798,1655,3611,1827,1784,3828,1907,1921,3603,1831,1772,3190,1670,1520,3574,1822,1752,3410,1692,1718,3448,1707,1741,3507,1751,1756,3785,1897,1888,3850,1917,1933,4007,2010,1997,3684,1828,1856,3333,1609,1724,3066,1407,1659,2146,959,1187,1320,593,727,1117,411,706,41.2,40.2,42.3,"Alabama, Blount",1009,0.189567


In [76]:
df = df[df['YEAR']==13][['FIPS', 'Name', 'AgeDist']]
# https://www.census.gov/programs-surveys/geography/technical-documentation/county-changes/2010.html
if df[df['FIPS'].isin([2063, 2066])].shape[0] == 2:
    age = df[df['FIPS']==2063]['AgeDist'].values + df[df['FIPS']==2066]['AgeDist'].values
    # additional_df = pd.DataFrame({'FIPS':2261, 'Name':'Alaska, Valdez-Cordova', 'Age': age[0]})
    df.loc[len(df.index)] = [2261, 'Alaska, Valdez-Cordova', age[0]] 
    # df = pd.concat([df, additional_df], ignore_index = True)
    df = df[~df['FIPS'].isin([2063, 2066])]

    df.sort_values(by=['FIPS'], inplace=True)

In [77]:
# PVI data came min max scaled
df['AgeDist'] = (df['AgeDist'] - df['AgeDist'].min()) / (df['AgeDist'].max() - df['AgeDist'].min())
df.round(4).to_csv(f'{output_folder}/Age Distribution.csv', index=False)
df.head()

,FIPS,Name,AgeDist
12,1001,"Alabama, Autauga",0.207972
26,1003,"Alabama, Baldwin",0.306861
40,1005,"Alabama, Barbour",0.283426
54,1007,"Alabama, Bibb",0.224280
68,1009,"Alabama, Blount",0.260583
